<a href="https://colab.research.google.com/github/subhashdevarasetty677/Q-A_Chatbot_locally_langchain_groq.ipynb/blob/main/Q%26A_Chatbot_locally_langchain_groq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit requests pyngrok langchain_groq langchain_huggingface langchain langchain_community python-dotenv langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 89.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 89.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 60.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 418.7/418.7 kB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 111.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 7

In [ ]:
!pip install langchain_community langchain_groq langchain_openai

In [ ]:
import os
import time
from langchain_groq import ChatGroq
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFDirectoryLoader
from dotenv import load_dotenv
from langchain.embeddings import HuggingFaceEmbeddings

In [ ]:
from google.colab import userdata
userdata.get('groq')

os.environ["GROQ_API_KEY"]=userdata.get('groq')
print("API Key Set Successfully!" if "GROQ_API_KEY" in os.environ else "Failed to Set API Key")

API Key Set Successfully!


In [ ]:
from google.colab import userdata
userdata.get('hgface')

os.environ["HF_API_TOKEN"]=userdata.get('hgface')
print("API Key Set Successfully!" if "HF_API_TOKEN" in os.environ else "Failed to Set API Key")

API Key Set Successfully!


In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [ ]:
prompt = ChatPromptTemplate.from_template(
    """
    please provide the answers based on query
    please provide accurate answers only based on the given prompt
    <context>
    {context}
    </context>
    Question: {input}
    """
)

In [ ]:
!pip install pypdf faiss-cpu

In [ ]:
def vector_embeddings(data_dir = "pdfs", max_documents=50):
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
  loader = PyPDFDirectoryLoader(data_dir)
  documents = loader.load()[:max_documents]
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
  texts = text_splitter.split_documents(documents)
  return FAISS.from_documents(texts, embeddings)

In [ ]:
vectors = vector_embeddings()

In [ ]:
llm = ChatGroq(model_name="llama3-8b-8192")

In [ ]:
input_text = input("Enter your question: ")
if input_text:
    document_chain = create_stuff_documents_chain(llm,prompt) #document chain is formed b/n llm and prompt
    retriever = vectors.as_retriever() # we called the vector embedding part
    chain = create_retrieval_chain(retriever,document_chain) #now entire chian pipeline is got built
    response = chain.invoke({"input": input_text})
    print(response['answer'])

Enter your question: what is health insurance coverage
According to the provided context, health insurance coverage is defined as:

"Comprehensive health insurance coverage status reflects an annual average of current comprehensive health insurance coverage status. This definition excludes single-service plans, such as accident, disability, dental, vision, or prescription medicine plans. Comprehensive health insurance covers basic health care needs."

In other words, health insurance coverage refers to having a plan that provides comprehensive coverage for basic health care needs, excluding single-service plans.


In [ ]:
#from langchain_core.output_parsers import StrOutputParser

In [ ]:

#output_parsers = StrOutputParser()
#chain=prompt | llm | output_parsers

#if input_text:
 #   response = chain.invoke({"question": input_text})
  #  st.write(response)